In [ ]:
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import pandas as pd
import requests
import math
import numpy
from datetime import datetime
from shapely import wkt
from shapely.errors import WKTReadingError
from shapely.geometry import mapping, shape
from shapely import Polygon


In [ ]:
gdf = gpd.read_file("./lotes_al_06_08.geojson")

In [ ]:
gdf

In [ ]:
gdf["datos.produccion"].loc[gdf["postgres_data.documento"] == 12190707].unique()

In [ ]:
from shapely.geometry import Polygon
import geopandas as gpd
import pandas as pd
import numpy as np

# Crear un DataFrame vacío para almacenar los resultados
resultados = []

# Crear un DataFrame para almacenar las categorías por productor
categoria_count = pd.DataFrame(columns=["documento", "productor", "categorias"])

# Establecer la prioridad de las categorías
categoria_prioridad = {
    '': 0,  # vacío
    'Qfield': 1,
    'Agritec': 2,
    'Mancipe': 3  # Aunque no usamos Mancipe, lo dejamos por si acaso
}

# Función para calcular el índice de forma
def indice_forma(polygon):
    if polygon.is_valid:
        perimeter = polygon.length
        area = polygon.area
        if perimeter == 0:
            return 1  # Caso raro de polígono degenerado
        return 4 * np.pi * area / (perimeter ** 2)
    return 0

# Iterar sobre los documentos únicos
for documento in gdf["postgres_data.documento"].unique():
    if documento:
        print(f"Evaluando documento: {documento}")

        gdf_filtro = gdf.loc[gdf["postgres_data.documento"] == documento]

        # Filtrar por productor
        for productor in gdf_filtro["datos.nombre_completo"].unique():
            gdf_filtro_productor = gdf_filtro.loc[gdf_filtro["datos.nombre_completo"] == productor]

            # Separar las categorías de los polígonos
            gdf_filtro_productor_null = gdf_filtro_productor[gdf_filtro_productor["datos.categoria"].isnull()]
            gdf_filtro_productor_qfield = gdf_filtro_productor[gdf_filtro_productor["datos.categoria"] == "Qfield"]
            gdf_filtro_productor_agritec = gdf_filtro_productor[gdf_filtro_productor["datos.categoria"] == "Agritec"]
            gdf_filtro_productor_mancipe = gdf_filtro_productor[gdf_filtro_productor["datos.categoria"] == "Mancipe"]

            # Concatenar las capas que se van a evaluar, sin los lotes de Mancipe
            gdf_filtro_productor_sorted = pd.concat([
                gdf_filtro_productor_null,
                gdf_filtro_productor_qfield,
                gdf_filtro_productor_agritec,
                gdf_filtro_productor_mancipe
            ])

            # Asegurar que estén en un CRS proyectado (metros)
            gdf_filtro_productor_sorted = gdf_filtro_productor_sorted.to_crs(epsg=3116)

            # Crear una columna para almacenar si el polígono es único (sin duplicados)
            gdf_filtro_productor_sorted["is_unique"] = True

            # Iterar para verificar duplicados basados en posición, tamaño y forma
            for idx, finca in gdf_filtro_productor_sorted.iterrows():
                finca_geom = finca.geometry
                finca_centroide = finca_geom.centroid  # Centroide del polígono
                finca_indice_forma = indice_forma(finca_geom)
                finca_area = finca_geom.area
                
                # Usar un margen de 5% para comparar geometrías
                margen_area = finca_area * 0.05

                # Comparar con el resto de polígonos
                for idx2, otra_finca in gdf_filtro_productor_sorted.drop(idx).iterrows():
                    otra_finca_geom = otra_finca.geometry
                    otra_finca_centroide = otra_finca_geom.centroid  # Centroide del otro polígono
                    otra_finca_indice_forma = indice_forma(otra_finca_geom)

                    # Comparar la distancia entre los centroides (posición)
                    distancia_centroides = finca_centroide.distance(otra_finca_centroide)

                    # Comparar el área (tamaño)
                    diferencia_area = abs(finca_area - otra_finca_geom.area)

                    # Comparar el índice de forma (similitud de la forma)
                    diferencia_forma = abs(finca_indice_forma - otra_finca_indice_forma)

                    # Si las geometrías están cerca en posición, tamaño y forma
                    if (distancia_centroides < 10 and  # Umbral de distancia entre centroides (ajustable)
                        diferencia_area < margen_area and  # Umbral de área (5%)
                        diferencia_forma < 0.05):  # Umbral de similitud de forma (ajustable)

                        categoria_finca = finca["datos.categoria"]
                        categoria_otra_finca = otra_finca["datos.categoria"]

                        # Si la categoría de la primera finca es más alta (prioritaria), se mantiene
                        if categoria_prioridad.get(categoria_finca, 0) >= categoria_prioridad.get(categoria_otra_finca, 0):
                            gdf_filtro_productor_sorted.at[idx2, "is_unique"] = False
                        else:
                            gdf_filtro_productor_sorted.at[idx, "is_unique"] = False
                        break

            # Filtrar los polígonos únicos (sin duplicados)
            gdf_filtro_productor_sorted_unique = gdf_filtro_productor_sorted[gdf_filtro_productor_sorted["is_unique"]]

            # Actualizar la columna `is_unique` en el dataset original
            for idx, finca in gdf_filtro_productor_sorted.iterrows():
                id_x = finca["id_x"]
                is_unique_value = finca["is_unique"]
                gdf.loc[gdf["id_x"] == id_x, "is_unique"] = is_unique_value

            # Inicializamos las variables de totales
            total_area_levante = 0
            total_plantas_levante = 0
            total_area_produccion = 0
            total_plantas_produccion = 0

            # Evaluar cada finca sin duplicados
            for idx, finca in gdf_filtro_productor_sorted_unique.iterrows():
                # Calcular área de la finca
                area_finca = finca.geometry.area

                # Filtrar los solapados que cumplen con el 5% de área
                gdf_solapado = gdf_filtro_productor_sorted_unique[
                    gdf_filtro_productor_sorted_unique.geometry.intersects(finca.geometry)
                ].copy()

                # Calcular área de intersección
                gdf_solapado["interseccion_area"] = gdf_solapado.geometry.intersection(finca.geometry).area

                # Filtrar los que tengan intersección significativa (>5% del área de la finca)
                gdf_solapado = gdf_solapado[gdf_solapado["interseccion_area"] > (0.05 * area_finca)]

                # Clasificar entre producción y levante
                gdf_filtro_productor_produccion = gdf_solapado[
                    gdf_solapado["datos.produccion"].isin(["Produccion", "produccion", "Producción", "producción", True, 'True'])
                ]
                gdf_filtro_productor_levante = gdf_solapado[
                    ~gdf_solapado["datos.produccion"].isin(["Produccion", "produccion", "Producción", "producción", True, 'True'])
                ]

                # Convertir a numérico y reemplazar NaN
                for df in [gdf_filtro_productor_produccion, gdf_filtro_productor_levante]:
                    df["area_x"] = pd.to_numeric(df["area_x"], errors='coerce').fillna(0)
                    df["datos.numero_plantas"] = pd.to_numeric(df["datos.numero_plantas"], errors='coerce').fillna(0)

                # Acumular áreas y plantas
                total_area_produccion += gdf_filtro_productor_produccion["area_x"].sum()
                total_plantas_produccion += gdf_filtro_productor_produccion["datos.numero_plantas"].sum()
                total_area_levante += gdf_filtro_productor_levante["area_x"].sum()
                total_plantas_levante += gdf_filtro_productor_levante["datos.numero_plantas"].sum()

            # Agregar resultados
            resultados.append({
                "documento": documento,
                "productor": productor,
                "area_levante": total_area_levante,
                "plantas_levante": total_plantas_levante,
                "area_produccion": total_area_produccion,
                "plantas_produccion": total_plantas_produccion,
            })



# Exportar resultados
resultados_df = pd.DataFrame(resultados)
resultados_df.to_excel("resultados_productores_con_mancipe.xlsx", index=False)
categoria_count.to_excel


In [ ]:
# Mapeamos los valores según lo que mencionas
gdf["datos.produccion"] = gdf["datos.produccion"].apply(lambda x: True if x in ['True', 'Produccion'] else False)

# Verificar los resultados
print(gdf["datos.produccion"].unique())


In [ ]:
# Rellenar los valores None con 'Qfield'
gdf["datos.categoria"] = gdf["datos.categoria"].fillna('Qfield')

# Verificar los resultados
print(gdf["datos.categoria"].unique())


In [ ]:
gdf.columns

In [ ]:
# Mapeamos los valores según lo que mencionas
gdf["datos.produccion"] = gdf["datos.produccion"].apply(lambda x: True if x in ['True', 'Produccion'] else False)

# Verificar los resultados
print(gdf["datos.produccion"].unique())


In [ ]:
resultado = []
for documento in gdf["postgres_data.documento"].unique():
    compilado = {}
    gdf_filtro = gdf.loc[gdf["postgres_data.documento"] == documento]
    compilado["documento"] = documento
    if not gdf_filtro.empty:
        # Asignar 'datos.nombre_completo' si no es nulo, de lo contrario 'datos.nombre_productor'
        compilado["nombre"] = gdf_filtro["datos.nombre_completo"].iloc[0] if pd.notnull(gdf_filtro["datos.nombre_completo"].iloc[0]) else gdf_filtro["datos.nombre_productor"].iloc[0]
    else:
        # Si gdf_filtro está vacío, asignar un valor por defecto o vacío
        compilado["nombre"] = ""

    for categoria in gdf_filtro["datos.categoria"].unique():
        gdf_filtro_categoria = gdf_filtro.loc[gdf_filtro["datos.categoria"] == categoria]
        
        # Verificar el tipo de 'datos.produccion'
        if gdf_filtro_categoria["datos.produccion"].dtype == bool:
            gdf_filtro_categoria_producccion = gdf_filtro_categoria.loc[gdf_filtro_categoria["datos.produccion"] == True]
            gdf_filtro_categoria_levante = gdf_filtro_categoria.loc[gdf_filtro_categoria["datos.produccion"] == False]
        else:
            # Si 'datos.produccion' está como string, usar "True" y "False"
            gdf_filtro_categoria_producccion = gdf_filtro_categoria.loc[gdf_filtro_categoria["datos.produccion"] == "True"]
            gdf_filtro_categoria_levante = gdf_filtro_categoria.loc[gdf_filtro_categoria["datos.produccion"] == "False"]
        
        compilado[f"{categoria}_area_levante"] = gdf_filtro_categoria_levante["area_x"].sum()
        compilado[f"{categoria}_plantas_levante"] = gdf_filtro_categoria_levante["datos.numero_plantas"].sum()
        compilado[f"{categoria}_area_produccion"] = gdf_filtro_categoria_producccion["area_x"].sum()
        compilado[f"{categoria}_plantas_produccion"] = gdf_filtro_categoria_producccion["datos.numero_plantas"].sum()

    #otros_categoria
    gdf_filtro_categoria = gdf_filtro.loc[gdf_filtro["datos.categoria"] != 'Mancipe']
    
    # Aplicar el mismo filtro para la producción
    if gdf_filtro_categoria["datos.produccion"].dtype == bool:
        gdf_filtro_categoria_producccion = gdf_filtro_categoria.loc[gdf_filtro_categoria["datos.produccion"] == True]
        gdf_filtro_categoria_levante = gdf_filtro_categoria.loc[gdf_filtro_categoria["datos.produccion"] == False]
    else:
        gdf_filtro_categoria_producccion = gdf_filtro_categoria.loc[gdf_filtro_categoria["datos.produccion"] == "True"]
        gdf_filtro_categoria_levante = gdf_filtro_categoria.loc[gdf_filtro_categoria["datos.produccion"] == "False"]
        
    compilado[f"otros_area_levante"] = gdf_filtro_categoria_levante["area_x"].sum()
    compilado[f"otros_plantas_levante"] = gdf_filtro_categoria_levante["datos.numero_plantas"].sum()
    compilado[f"otros_area_produccion"] = gdf_filtro_categoria_producccion["area_x"].sum()
    compilado[f"otros_plantas_produccion"] = gdf_filtro_categoria_producccion["datos.numero_plantas"].sum()

    resultado.append(compilado)


In [ ]:
df = pd.DataFrame(resultado)

In [ ]:
df

In [ ]:
df.to_excel("./habilidad 08 08.xlsx")

In [ ]:
pequeño_analisis = gdf.loc[gdf["postgres_data.documento"] == 1082155361]
pequeño_analisis["area_x"].loc[pequeño_analisis["datos.produccion"]].sum()

In [ ]:
# Asegurarnos de que 'datos.numero_plantas' sea un número válido

def limpiar_numero_plantas(valor):
    if pd.isnull(valor):
        return 0
    try:
        # Si es un número válido, lo devuelve directamente
        return float(valor)
    except:
        # Si está mal formateado (por ejemplo, '4785.01632.01150.0'), intenta rescatar el primer número
        # y descarta el resto para evitar inflar los conteos
        partes = str(valor).split(".")
        if partes and partes[0].isdigit():
            return float(partes[0])
        return 0

# Aplicar la función al GeoDataFrame
gdf['datos.numero_plantas'] = gdf['datos.numero_plantas'].apply(limpiar_numero_plantas).astype(int)


In [ ]:
import pandas as pd
import geopandas as gpd

# Función para asignar la categoría priorizada
def asignar_categoria(row):
    if row['datos.categoria'] == 'Mancipe':
        return 'Mancipe'
    elif row['datos.categoria'] == 'Agritech':
        return 'Agritech'
    elif row['datos.categoria'] == 'Qfield':
        return 'Qfield'
    else:
        return 'Otros'

# Función para limpiar los valores en datos.numero_plantas
def limpiar_numero_plantas(valor):
    if pd.isnull(valor):
        return 0
    try:
        return int(float(valor))
    except:
        partes = str(valor).split(".")
        if partes and partes[0].isdigit():
            return int(partes[0])
        return 0

# Aplicar limpieza
gdf['datos.numero_plantas'] = gdf['datos.numero_plantas'].apply(limpiar_numero_plantas)

# Asignar categoría priorizada
gdf['categoria_priorizada'] = gdf.apply(asignar_categoria, axis=1)

# Filtrar las categorías relevantes (excluyendo Mancipe)
gdf_filtrado = gdf[gdf['categoria_priorizada'] != 'Mancipe']

# Usamos area_x directamente
gdf_filtrado['area'] = pd.to_numeric(gdf_filtrado['area_x'], errors='coerce').fillna(0)

# Inicializar resultados
resultados = []

# Iterar sobre los productores
for productor in gdf_filtrado['datos.nombre_completo'].unique():
    gdf_productor = gdf_filtrado[gdf_filtrado['datos.nombre_completo'] == productor]

    # Obtener el documento del productor (asumimos que es único por productor)
    documento = gdf_productor['postgres_data.documento'].iloc[0] if not gdf_productor['postgres_data.documento'].isnull().all() else None

    # Inicializar diccionario de totales
    totals = {
        'documento': documento,
        'productor': productor,
        'Agritech_area_levante': 0,
        'Agritech_plantas_levante': 0,
        'Agritech_area_produccion': 0,
        'Agritech_plantas_produccion': 0,
        'Qfield_area_levante': 0,
        'Qfield_plantas_levante': 0,
        'Qfield_area_produccion': 0,
        'Qfield_plantas_produccion': 0,
        'Otros_area_levante': 0,
        'Otros_plantas_levante': 0,
        'Otros_area_produccion': 0,
        'Otros_plantas_produccion': 0
    }

    for categoria in ['Agritech', 'Qfield', 'Otros']:
        gdf_categoria = gdf_productor[gdf_productor['categoria_priorizada'] == categoria]

        # Clasificar por tipo de producción
        gdf_levante = gdf_categoria[~gdf_categoria['datos.produccion'].isin(["Produccion", "produccion", "Producción", "producción", True])]
        gdf_produccion = gdf_categoria[gdf_categoria['datos.produccion'].isin(["Produccion", "produccion", "Producción", "producción", True])]

        # Sumar valores
        totals[f'{categoria}_area_levante'] = gdf_levante['area'].sum()
        totals[f'{categoria}_plantas_levante'] = gdf_levante['datos.numero_plantas'].sum()
        totals[f'{categoria}_area_produccion'] = gdf_produccion['area'].sum()
        totals[f'{categoria}_plantas_produccion'] = gdf_produccion['datos.numero_plantas'].sum()

    resultados.append(totals)

# Crear DataFrame final
resultados_df = pd.DataFrame(resultados)

# Exportar a Excel
resultados_df.to_excel('resultados_productores_por_categoria.xlsx', index=False)

print("✅ Análisis completado y resultados exportados.")


In [ ]:
gdf = gpd.read_file("./Fincas_al_06_08.geojson")

In [ ]:
gdf.columns

In [ ]:

compilado = []

for cedula in gdf["postgres_data.documento_x"].unique():
    # Filtramos por cedula
    gdf_filtro = gdf.loc[gdf["postgres_data.documento_x"] == cedula]
    
    try: 
        nombre = gdf_filtro["datos.nombre_completo_x"].iloc[0]
    except:
        nombre = None
    # Unificamos geometrías (para evitar solapes)
    union_geom = gdf_filtro.unary_union
    
    # Creamos un GeoSeries temporal para reproyectar y calcular área
    gdf_union = gpd.GeoSeries([union_geom], crs=gdf.crs).to_crs(epsg=3116)
    
    # Área en metros cuadrados
    area_m2 = gdf_union.area.iloc[0]
    
    compilado.append({
        "cedula": cedula,
        "nombre": nombre,
        "area_ha": area_m2/10000
    })
df_reporte = pd.DataFrame(compilado)

In [ ]:
df_reporte.to_excel("./fincas_al_06_08.xlsx")